In [195]:
import tensorflow as tf
from keras.layers import Input, Lambda,subtract,GlobalMaxPooling2D,Dense,GlobalAveragePooling2D, concatenate, Activation
from keras.applications.mobilenet import MobileNet as Net
import scipy.sparse as sparse
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.layers import Input, merge
from keras.optimizers import Adam
import cv2
import numpy as np
import os
from tqdm import tqdm
from keras.models import Sequential
import numpy as np
import pandas as pd
from numpy import linalg as lin_al
from sklearn.linear_model import Ridge
import numpy as np
import sklearn.datasets
#import sklearn.cross_validation
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn import ensemble
import time
import pickle
import random
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import keras 
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.preprocessing import OneHotEncoder
import re
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder
from keras.layers import RepeatVector
from keras.layers import Permute
from keras.layers import merge
from keras.layers import Lambda
from keras.layers import multiply
from keras.layers import concatenate
from keras.layers import Convolution1D
from keras.layers import MaxPooling1D
from keras.layers import GlobalAveragePooling1D
from keras.layers import GlobalMaxPooling1D
from sklearn import preprocessing
import tensorflow as tf
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import implicit
import csv
from scipy.spatial.distance import cosine as cs
from keras.layers import Merge
from keras.layers import dot
from keras.constraints import non_neg


In [2]:
train=pd.read_csv('/Users/s0c02nj/Desktop/train- Capillary 4/train.csv')
product_attr= pd.read_csv('/Users/s0c02nj/Desktop/train- Capillary 4/product_attributes.csv')
test =pd.read_csv('/Users/s0c02nj/Desktop/train- Capillary 4/test_nFNPSyV.csv')
sample_sub= pd.read_csv('/Users/s0c02nj/Desktop/train- Capillary 4/sample_submission_qfCnaKZ.csv')

In [3]:
len(train['UserId'].unique())

27778

In [4]:
len(train['productid'].unique())

3026

In [5]:
#Implementing the Deep Learning Implementation

In [11]:
user_df=pd.DataFrame(train['UserId'].unique())
user_df.columns=['user_id']

In [12]:
item_df=pd.DataFrame(train['productid'].unique())
item_df.columns=['item_id']

In [13]:
cart_data=user_df.assign(foo=1).merge(item_df.assign(foo=1)).drop('foo', 1)
cart_data.head()

,user_id,item_id
0,18075,12322648
1,18075,12371370
2,18075,12973004
3,18075,12657560
4,18075,11659914


In [24]:
#Joining to make the binary class
train_join= pd.merge(left=cart_data, right=train, how='left', left_on=['user_id','item_id'], right_on=['UserId','productid'])


In [35]:
#train_join=train_join.drop(['productid','OrderDate'],axis=1)
#train_join=train_join.drop(['UserId'],axis=1)

In [40]:
#Replace Nulls with 0
train_join['Quantity'].fillna(0, inplace=True)

In [58]:
join1= train_join[train_join['Quantity']==0]
join2 = train_join[train_join['Quantity']!=0]

In [60]:
join2.head()

,user_id,item_id,Quantity
0,18075,12322648,1.0
1,18075,12322648,1.0
31,18075,11659734,1.0
53,18075,12407770,1.0
79,18075,12658298,1.0


In [61]:
#Replacing with all 1s
join2['Quantity']=1

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [65]:
join1_1= join1.sample(n=100000)

In [67]:
final_data=join1_1.append(join2)
final_data.index=range(0,len(final_data))

In [70]:
#final_data is the binary class
final_data.head()

,user_id,item_id,Quantity
0,16520,11148888,0.0
1,11896,12407510,0.0
2,2437,13039450,0.0
3,13066,11659740,0.0
4,2847,12936744,0.0


In [74]:
#Cat Encoding for User id
from sklearn.preprocessing import LabelEncoder
le_user= LabelEncoder()
user_id_enc = le_user.fit_transform(final_data['user_id'])

In [75]:
#Cat Encoding for Item id
le_item=LabelEncoder()
item_id_enc = le_item.fit_transform(final_data['item_id'])

In [76]:
n_users= len(final_data['user_id'].unique())
n_items= len(final_data['item_id'].unique())

In [79]:
#Extracting the product attributes


In [80]:
#Product Mapping Dictionary
prod_dict = {}
for i in range(0,len(product_attr)):
    prod_dict[(str(product_attr.iloc[i]['productid']),product_attr.iloc[i]['attribute_name'])] = product_attr.iloc[i]['attributevalue']


In [84]:
product_attr.head()

,productid,attribute_name,attributevalue
0,11145600,Fit,37
1,11145600,Sleeve Length,23
2,11145600,Fabric,16
3,11145600,Color,25
4,11145600,Neckline,51


In [88]:
len(train['productid'].unique())

3026

In [89]:
item_matrix=pd.DataFrame(np.zeros((3026,9)))

item_matrix.columns= ['p_id', 'Category', 'Fit', 'Sleeve Length', 'Fabric', 'Color',
       'Season', 'Collection', 'Neckline']

item_matrix['p_id']=train['productid'].unique()

In [91]:
item_matrix.head()

,p_id,Category,Fit,Sleeve Length,Fabric,Color,Season,Collection,Neckline
0,12322648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12371370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12973004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12657560,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11659914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
#Item df
def get_item_df(str_name):
    df=[]
    k=0
    for i in range(0,len(item_matrix)):
        #print i
        key_pair= str(item_matrix['p_id'].iloc[i]),str_name
        if key_pair in prod_dict.keys():
            df.append(prod_dict[key_pair])
        else :
            df.append(k)    
            
    return df

In [115]:
#Defining the Item Attributes
category = get_item_df('Category')
fit = get_item_df('Fit')
slv_len = get_item_df('Sleeve Length' )
fabric= get_item_df('Fabric')
color = get_item_df ('Color')
season = get_item_df ('Season')
collection = get_item_df ('Collection')
neckline = get_item_df('Neckline')

In [116]:
#Item Matrix
item_matrix['Category']= category
item_matrix['Fit']= fit
item_matrix['Sleeve Length']= slv_len
item_matrix['Fabric']= fabric
item_matrix['Color']= color
item_matrix['Season']= season
item_matrix['Collection']= collection
item_matrix['Neckline']= neckline

In [121]:
#Joining to make the binary class
df_with_item_att= pd.merge(left=final_data, right=item_matrix, how='inner', left_on=['item_id'], right_on=['p_id'])

In [127]:
df_item_att= df_with_item_att[['Category','Fit','Sleeve Length','Fabric','Color','Season','Collection','Neckline']]
#df_item_id= df_with_item_att['item_id']
#df_user_id = df_with_item_att['user_id']

In [131]:
df_item_att.shape

(232551, 8)

In [129]:
y=df_with_item_att['Quantity']
y_cat=to_categorical(y,num_classes=2)

In [ ]:
#Performing the Model

In [282]:
def model_recco():
    
    #Defining the input-----> Item
    item_input1 = Input(shape=(1,))
    item_embedding = Embedding(n_items ,50 ,input_length=1,name='Item-Embedding')(item_input1)
    item_vec = Flatten(name='FlattenEmbed_Item')(item_embedding)
    
    #Adding the Input Desc Information
    item_input2 = Input(shape=(8,))
    layer_dense = Dense(10, activation='relu')(item_input2)
    
    #Item Image Info
    
    #Merging the Item Information
    merged_item = concatenate([item_vec,layer_dense],axis=1)
    item_embed =  Dense(40, activation='relu',name='Item-Vec')(merged_item)
    
    #Defining the input-------> User
    user_input = Input(shape=(1,))
    user_embedding = Embedding(n_users ,50 ,input_length=1,name='User-Embedding')(user_input)
    user_vec = Flatten(name='FlattenEmbed_User')(user_embedding)
    user_embed = Dense(40, activation='relu',name='User-Vec')(user_vec)
    
    #Merge Layer
    final_layer = concatenate([item_embed,user_embed],axis=1)
    
    #Output_Layer
    final_output = Dense(2, activation='softmax')(final_layer)
    
    #Model
    model = Model([user_input, item_input1,item_input2], outputs=final_output)
    
    return model

    

In [283]:
model=model_recco()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
Item-Embedding (Embedding)      (None, 1, 50)        151300      input_16[0][0]                   
__________________________________________________________________________________________________
input_17 (InputLayer)           (None, 8)            0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
FlattenEmb

In [284]:
model.compile(loss = 'binary_crossentropy', optimizer = Adam(lr=0.001), metrics = ["accuracy"])

In [285]:
history = model.fit([user_id_enc, item_id_enc,df_item_att], 
                    y_cat, batch_size = 512, 
                    validation_split=0.1, class_weight = {0:0.3 ,1:0.7},
                    epochs=3, verbose=1)


Train on 209295 samples, validate on 23256 samples
Epoch 1/3
209295/209295 [==============================] - ETA: 0s - loss: 0.2776 - acc: 0.617 - 8s 38us/step - loss: 0.2774 - acc: 0.6181 - val_loss: 0.2850 - val_acc: 0.6449
Epoch 2/3
209295/209295 [==============================] - 8s 37us/step - loss: 0.2336 - acc: 0.6865 - val_loss: 0.3041 - val_acc: 0.6268
Epoch 3/3
209295/209295 [==============================] - 8s 37us/step - loss: 0.2181 - acc: 0.7171 - val_loss: 0.3287 - val_acc: 0.6136


In [286]:
user_embedding_learnt = model.get_layer(name='User-Embedding').get_weights()[0]

In [287]:
item_embedding_learnt = model.get_layer(name='Item-Embedding').get_weights()[0]

In [288]:
user_embedding_learnt.shape

(27778, 50)

In [166]:
#Getting back the original ids
user_id_inv= le_user.inverse_transform(np.unique(user_id_enc))
item_id_inv= le_item.inverse_transform(np.unique(item_id_enc))

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [168]:
#Item final embed matrix
item_emed_df= pd.DataFrame(item_embedding_learnt)
item_emed_df['item_id']= item_id_inv

In [169]:
#User Embed matrix
user_emed_df= pd.DataFrame(user_embedding_learnt)
user_emed_df['user_id']= user_id_inv

In [171]:
#user_emed_df.head()

In [172]:
#Joining 
test_join= pd.merge(left=test, right=user_emed_df, how='inner', left_on=['UserId'], right_on=['user_id'])


In [176]:
test_join=test_join.drop(['user_id'],axis=1)
#test_join.head()

In [178]:
test_join.shape

(2350, 50)

In [199]:
test_join.shape

(2350, 50)

In [202]:
item_emed_copy= item_emed_df.drop(['item_id'],axis=1)

In [204]:
#Transpose
item_emed_copy=item_emed_copy.transpose()

In [251]:
#Mat_mult
final_mat =  np.matmul(test_join, item_emed_copy)


In [263]:
k2=np.argsort(final_mat[1])[-10:]
k2

array([ 176, 1834, 2615, 2191, 1816, 2062, 2940, 1433, 2742, 1328])

In [264]:
k2[::-1]

array([1328, 2742, 1433, 2940, 2062, 1816, 2191, 2615, 1834,  176])

In [265]:
item_emed_df['item_id'].iloc[k2]

176     11360778
1834    12657562
2615    13039456
2191    12658462
1816    12657500
2062    12658186
2940    13653590
1433    12407648
2742    13039792
1328    12407382
Name: item_id, dtype: int64

In [198]:
# sim=[]
# final_list=[]
# for i in range(0,len(test['UserId'])):
#     print i
#     for j in range(0,len(item_emed_df)):
#         sim.append(cs(test_join.iloc[i],item_emed_df.iloc[j][0:100]))
#     p=np.argsort(sim)[-10:]
#     final_list.append(p)?
#     sim=[]

In [266]:
final=[]
for i in range(0,len(final_mat)):
    k2=np.argsort(final_mat[i])[-10:]
    k2=k2[::-1]
    final.append(list(item_emed_df['item_id'].iloc[k2]))

In [262]:
#product_attr[product_attr['productid']==11660064]

In [272]:
#train[train['UserId']==28]

In [275]:
sample_sub['product_list']=final
sample_sub.head()

,UserId,product_list
0,2,"[12407648, 12407382, 13039792, 13653590, 12658..."
1,28,"[12407382, 13039792, 12407648, 13653590, 12658..."
2,36,"[12407382, 12407648, 13039792, 12658634, 13653..."
3,38,"[13039792, 12658462, 12407648, 12407382, 13653..."
4,41,"[12407648, 13039792, 13038826, 13653590, 12407..."


In [277]:
sample_sub.to_csv('/Users/s0c02nj/Desktop/Submission_Cap.csv',index=False)